## 数据集分割方法【欢迎同学们一起交流总结方法，共同进步，做大做强，有好的方法或者建议，欢迎邮件765187944@qq.com】

In [1]:
# 创建数据集
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import os
import pandas as pd

## 创建映射型数据集

In [3]:
class MyData(Dataset):
    def __init__(self, image_dir, label_dir=None, transform=None):
        super(MyData, self).__init__()
        self.image_dir = image_dir
        self.transform = transform
        if label_dir:
            self.df = pd.read_csv(label_dir)
            self.label_dict = {}
            for i, label in enumerate(self.df['label'].unique()):
                self.label_dict[label]=i
            self.reverse_label_dict = {v:k for k,v in self.label_dict.items()}
            self.df['encoded'] = self.df['label'].map(self.label_dict)
        else:
            self.df = pd.DataFrame({'image': os.listdir(self.image_dir),
                                   'label': np.zeros(len(os.listdir(self.image_dir))),
                                   'encoded': np.zeros(len(os.listdir(self.image_dir)))})
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, item):
        image_path = os.path.join(self.image_dir, self.df.iloc[item, 0])
        image = np.asarray(Image.open(image_path).convert('RGB'))
        label = torch.tensor(self.df.iloc[item, 2])
        if self.transform:
            image = self.transform(image)
        return image, label
        
            
    

In [4]:
train_valid_data = MyData(image_dir='E:\\Mlearning\\DL_competition\\classify leaves\\train_set', label_dir = 'E:\\Mlearning\\DL_competition\\classify leaves\\train.csv')

In [5]:
len(train_valid_data)

18353

## 分割数据集【Subset and random_split】
## subset与random_split直接针对数据集（image与label都被划分）划分

In [5]:
# Subset and np.random.shuffle
data_size = len(train_valid_data)
data_idx = list(range(data_size))
np.random.shuffle(data_idx)
valid_size = int(np.floor(data_size*0.2))
train_idx = data_idx[valid_size:]
valid_idx = data_idx[:valid_size]
train_data = torch.utils.data.Subset(train_valid_data, train_idx)
valid_data = torch.utils.data.Subset(train_valid_data, valid_idx)

In [6]:
len(train_data), len(valid_data)

(14683, 3670)

In [7]:
# torch.utils.data.random_split
train_set, valid_set = torch.utils.data.random_split(train_valid_data, lengths=[18000, 353], generator=torch.Generator().manual_seed(0))
len(train_set)

18000

##  sampler取样，取样是先划分索引，再在数据集中取对应的Image与label【与上面方法对比记忆】

In [24]:
# sampler and SubsetRandomSampler[shuffle方式采用np.random.shuffle]
train_size = len(train_valid_data)
data_idx = list(range(train_size))
np.random.seed(42)
np.random.shuffle(data_idx)
split = int(np.floor(train_size*0.2))
train_idx, valid_idx = data_idx[split:], data_idx[:split]
train_sampler = torch.utils.data.RandomSampler(train_idx)
valid_sampler = torch.utils.data.RandomSampler(valid_idx)
train_loader = torch.utils.data.DataLoader(train_valid_data, batch_size=128, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_valid_data, batch_size=128, sampler=valid_sampler)

## 权重取样

In [12]:
test_data = train_valid_data
classes, counts = np.unique(test_data.df['label'], return_counts=True)
num_class = classes.shape[0]
count = counts.max()
weights = np.zeros(len(test_data))
for c in classes:
    freq_c = np.sum(test_data.df['label']==c)
    weights[test_data.df['label']==c] = len(test_data)/freq_c
weights = list(weights)
sampler = torch.utils.data.WeightedRandomSampler(weights, int(num_class*count))
len(sampler)

62128

## sklearn 数据分割方法【建议与pytorch不同目标划分对应记忆】

In [29]:
## train_test_split【针对数据集划分】
from sklearn.model_selection import train_test_split
train_set, valid_set = train_test_split(train_valid_data, test_size=0.2)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
len(train_loader)

115

## 交叉验证

In [31]:
## KFOLD交叉验证+sampler[SubsetRandomSampler, RandomSampler,WeightedRandomSampler]【针对索引划分后在数据集中取样】
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
for train_idx, valid_idx in kfold.split(train_valid_data):
    print(train_idx, valid_idx)

[ 3671  3672  3673 ... 18350 18351 18352] [   0    1    2 ... 3668 3669 3670]
[    0     1     2 ... 18350 18351 18352] [3671 3672 3673 ... 7339 7340 7341]
[    0     1     2 ... 18350 18351 18352] [ 7342  7343  7344 ... 11010 11011 11012]
[    0     1     2 ... 18350 18351 18352] [11013 11014 11015 ... 14680 14681 14682]
[    0     1     2 ... 14680 14681 14682] [14683 14684 14685 ... 18350 18351 18352]


In [33]:
## RepeatedKFold+sampler[SubsetRandomSampler, RandomSampler, WeightedRandomSampler]
from sklearn.model_selection import RepeatedKFold
rkfd = RepeatedKFold(n_splits=5, n_repeats=2)
for train_idx, valid_idx in rkfd.split(train_valid_data):
    print(train_idx, valid_idx)

[    0     1     2 ... 18349 18351 18352] [    4    18    24 ... 18343 18346 18350]
[    0     1     2 ... 18349 18350 18352] [    6     8    14 ... 18338 18348 18351]
[    1     3     4 ... 18350 18351 18352] [    0     2    12 ... 18336 18347 18349]
[    0     1     2 ... 18350 18351 18352] [    3     5     7 ... 18324 18329 18344]
[    0     2     3 ... 18349 18350 18351] [    1    10    13 ... 18342 18345 18352]
[    0     1     2 ... 18349 18351 18352] [   10    14    24 ... 18332 18339 18350]
[    0     2     4 ... 18349 18350 18351] [    1     3     9 ... 18345 18348 18352]
[    1     2     3 ... 18350 18351 18352] [    0     4     7 ... 18338 18340 18349]
[    0     1     2 ... 18350 18351 18352] [    5     6    12 ... 18342 18343 18346]
[    0     1     3 ... 18349 18350 18352] [    2     8    19 ... 18333 18347 18351]


In [38]:
## leave one/p out
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
for i, (train_idx, valid_idx) in enumerate(loo.split(train_valid_data)):
    print(i, train_idx, valid_idx)

0 [    1     2     3 ... 18350 18351 18352] [0]
1 [    0     2     3 ... 18350 18351 18352] [1]
2 [    0     1     3 ... 18350 18351 18352] [2]
3 [    0     1     2 ... 18350 18351 18352] [3]
4 [    0     1     2 ... 18350 18351 18352] [4]
5 [    0     1     2 ... 18350 18351 18352] [5]
6 [    0     1     2 ... 18350 18351 18352] [6]
7 [    0     1     2 ... 18350 18351 18352] [7]
8 [    0     1     2 ... 18350 18351 18352] [8]
9 [    0     1     2 ... 18350 18351 18352] [9]
10 [    0     1     2 ... 18350 18351 18352] [10]
11 [    0     1     2 ... 18350 18351 18352] [11]
12 [    0     1     2 ... 18350 18351 18352] [12]
13 [    0     1     2 ... 18350 18351 18352] [13]
14 [    0     1     2 ... 18350 18351 18352] [14]
15 [    0     1     2 ... 18350 18351 18352] [15]
16 [    0     1     2 ... 18350 18351 18352] [16]
17 [    0     1     2 ... 18350 18351 18352] [17]
18 [    0     1     2 ... 18350 18351 18352] [18]
19 [    0     1     2 ... 18350 18351 18352] [19]
20 [    0     1     

261 [    0     1     2 ... 18350 18351 18352] [261]
262 [    0     1     2 ... 18350 18351 18352] [262]
263 [    0     1     2 ... 18350 18351 18352] [263]
264 [    0     1     2 ... 18350 18351 18352] [264]
265 [    0     1     2 ... 18350 18351 18352] [265]
266 [    0     1     2 ... 18350 18351 18352] [266]
267 [    0     1     2 ... 18350 18351 18352] [267]
268 [    0     1     2 ... 18350 18351 18352] [268]
269 [    0     1     2 ... 18350 18351 18352] [269]
270 [    0     1     2 ... 18350 18351 18352] [270]
271 [    0     1     2 ... 18350 18351 18352] [271]
272 [    0     1     2 ... 18350 18351 18352] [272]
273 [    0     1     2 ... 18350 18351 18352] [273]
274 [    0     1     2 ... 18350 18351 18352] [274]
275 [    0     1     2 ... 18350 18351 18352] [275]
276 [    0     1     2 ... 18350 18351 18352] [276]
277 [    0     1     2 ... 18350 18351 18352] [277]
278 [    0     1     2 ... 18350 18351 18352] [278]
279 [    0     1     2 ... 18350 18351 18352] [279]
280 [    0  

525 [    0     1     2 ... 18350 18351 18352] [525]
526 [    0     1     2 ... 18350 18351 18352] [526]
527 [    0     1     2 ... 18350 18351 18352] [527]
528 [    0     1     2 ... 18350 18351 18352] [528]
529 [    0     1     2 ... 18350 18351 18352] [529]
530 [    0     1     2 ... 18350 18351 18352] [530]
531 [    0     1     2 ... 18350 18351 18352] [531]
532 [    0     1     2 ... 18350 18351 18352] [532]
533 [    0     1     2 ... 18350 18351 18352] [533]
534 [    0     1     2 ... 18350 18351 18352] [534]
535 [    0     1     2 ... 18350 18351 18352] [535]
536 [    0     1     2 ... 18350 18351 18352] [536]
537 [    0     1     2 ... 18350 18351 18352] [537]
538 [    0     1     2 ... 18350 18351 18352] [538]
539 [    0     1     2 ... 18350 18351 18352] [539]
540 [    0     1     2 ... 18350 18351 18352] [540]
541 [    0     1     2 ... 18350 18351 18352] [541]
542 [    0     1     2 ... 18350 18351 18352] [542]
543 [    0     1     2 ... 18350 18351 18352] [543]
544 [    0  

785 [    0     1     2 ... 18350 18351 18352] [785]
786 [    0     1     2 ... 18350 18351 18352] [786]
787 [    0     1     2 ... 18350 18351 18352] [787]
788 [    0     1     2 ... 18350 18351 18352] [788]
789 [    0     1     2 ... 18350 18351 18352] [789]
790 [    0     1     2 ... 18350 18351 18352] [790]
791 [    0     1     2 ... 18350 18351 18352] [791]
792 [    0     1     2 ... 18350 18351 18352] [792]
793 [    0     1     2 ... 18350 18351 18352] [793]
794 [    0     1     2 ... 18350 18351 18352] [794]
795 [    0     1     2 ... 18350 18351 18352] [795]
796 [    0     1     2 ... 18350 18351 18352] [796]
797 [    0     1     2 ... 18350 18351 18352] [797]
798 [    0     1     2 ... 18350 18351 18352] [798]
799 [    0     1     2 ... 18350 18351 18352] [799]
800 [    0     1     2 ... 18350 18351 18352] [800]
801 [    0     1     2 ... 18350 18351 18352] [801]
802 [    0     1     2 ... 18350 18351 18352] [802]
803 [    0     1     2 ... 18350 18351 18352] [803]
804 [    0  

1050 [    0     1     2 ... 18350 18351 18352] [1050]
1051 [    0     1     2 ... 18350 18351 18352] [1051]
1052 [    0     1     2 ... 18350 18351 18352] [1052]
1053 [    0     1     2 ... 18350 18351 18352] [1053]
1054 [    0     1     2 ... 18350 18351 18352] [1054]
1055 [    0     1     2 ... 18350 18351 18352] [1055]
1056 [    0     1     2 ... 18350 18351 18352] [1056]
1057 [    0     1     2 ... 18350 18351 18352] [1057]
1058 [    0     1     2 ... 18350 18351 18352] [1058]
1059 [    0     1     2 ... 18350 18351 18352] [1059]
1060 [    0     1     2 ... 18350 18351 18352] [1060]
1061 [    0     1     2 ... 18350 18351 18352] [1061]
1062 [    0     1     2 ... 18350 18351 18352] [1062]
1063 [    0     1     2 ... 18350 18351 18352] [1063]
1064 [    0     1     2 ... 18350 18351 18352] [1064]
1065 [    0     1     2 ... 18350 18351 18352] [1065]
1066 [    0     1     2 ... 18350 18351 18352] [1066]
1067 [    0     1     2 ... 18350 18351 18352] [1067]
1068 [    0     1     2 ... 

...

17150 [    0     1     2 ... 18350 18351 18352] [17150]
17151 [    0     1     2 ... 18350 18351 18352] [17151]
17152 [    0     1     2 ... 18350 18351 18352] [17152]
17153 [    0     1     2 ... 18350 18351 18352] [17153]
17154 [    0     1     2 ... 18350 18351 18352] [17154]
17155 [    0     1     2 ... 18350 18351 18352] [17155]
17156 [    0     1     2 ... 18350 18351 18352] [17156]
17157 [    0     1     2 ... 18350 18351 18352] [17157]
17158 [    0     1     2 ... 18350 18351 18352] [17158]
17159 [    0     1     2 ... 18350 18351 18352] [17159]
17160 [    0     1     2 ... 18350 18351 18352] [17160]
17161 [    0     1     2 ... 18350 18351 18352] [17161]
17162 [    0     1     2 ... 18350 18351 18352] [17162]
17163 [    0     1     2 ... 18350 18351 18352] [17163]
17164 [    0     1     2 ... 18350 18351 18352] [17164]
17165 [    0     1     2 ... 18350 18351 18352] [17165]
17166 [    0     1     2 ... 18350 18351 18352] [17166]
17167 [    0     1     2 ... 18350 18351 18352] 

17419 [    0     1     2 ... 18350 18351 18352] [17419]
17420 [    0     1     2 ... 18350 18351 18352] [17420]
17421 [    0     1     2 ... 18350 18351 18352] [17421]
17422 [    0     1     2 ... 18350 18351 18352] [17422]
17423 [    0     1     2 ... 18350 18351 18352] [17423]
17424 [    0     1     2 ... 18350 18351 18352] [17424]
17425 [    0     1     2 ... 18350 18351 18352] [17425]
17426 [    0     1     2 ... 18350 18351 18352] [17426]
17427 [    0     1     2 ... 18350 18351 18352] [17427]
17428 [    0     1     2 ... 18350 18351 18352] [17428]
17429 [    0     1     2 ... 18350 18351 18352] [17429]
17430 [    0     1     2 ... 18350 18351 18352] [17430]
17431 [    0     1     2 ... 18350 18351 18352] [17431]
17432 [    0     1     2 ... 18350 18351 18352] [17432]
17433 [    0     1     2 ... 18350 18351 18352] [17433]
17434 [    0     1     2 ... 18350 18351 18352] [17434]
17435 [    0     1     2 ... 18350 18351 18352] [17435]
17436 [    0     1     2 ... 18350 18351 18352] 

17676 [    0     1     2 ... 18350 18351 18352] [17676]
17677 [    0     1     2 ... 18350 18351 18352] [17677]
17678 [    0     1     2 ... 18350 18351 18352] [17678]
17679 [    0     1     2 ... 18350 18351 18352] [17679]
17680 [    0     1     2 ... 18350 18351 18352] [17680]
17681 [    0     1     2 ... 18350 18351 18352] [17681]
17682 [    0     1     2 ... 18350 18351 18352] [17682]
17683 [    0     1     2 ... 18350 18351 18352] [17683]
17684 [    0     1     2 ... 18350 18351 18352] [17684]
17685 [    0     1     2 ... 18350 18351 18352] [17685]
17686 [    0     1     2 ... 18350 18351 18352] [17686]
17687 [    0     1     2 ... 18350 18351 18352] [17687]
17688 [    0     1     2 ... 18350 18351 18352] [17688]
17689 [    0     1     2 ... 18350 18351 18352] [17689]
17690 [    0     1     2 ... 18350 18351 18352] [17690]
17691 [    0     1     2 ... 18350 18351 18352] [17691]
17692 [    0     1     2 ... 18350 18351 18352] [17692]
17693 [    0     1     2 ... 18350 18351 18352] 

17936 [    0     1     2 ... 18350 18351 18352] [17936]
17937 [    0     1     2 ... 18350 18351 18352] [17937]
17938 [    0     1     2 ... 18350 18351 18352] [17938]
17939 [    0     1     2 ... 18350 18351 18352] [17939]
17940 [    0     1     2 ... 18350 18351 18352] [17940]
17941 [    0     1     2 ... 18350 18351 18352] [17941]
17942 [    0     1     2 ... 18350 18351 18352] [17942]
17943 [    0     1     2 ... 18350 18351 18352] [17943]
17944 [    0     1     2 ... 18350 18351 18352] [17944]
17945 [    0     1     2 ... 18350 18351 18352] [17945]
17946 [    0     1     2 ... 18350 18351 18352] [17946]
17947 [    0     1     2 ... 18350 18351 18352] [17947]
17948 [    0     1     2 ... 18350 18351 18352] [17948]
17949 [    0     1     2 ... 18350 18351 18352] [17949]
17950 [    0     1     2 ... 18350 18351 18352] [17950]
17951 [    0     1     2 ... 18350 18351 18352] [17951]
17952 [    0     1     2 ... 18350 18351 18352] [17952]
17953 [    0     1     2 ... 18350 18351 18352] 

18146 [    0     1     2 ... 18350 18351 18352] [18146]
18147 [    0     1     2 ... 18350 18351 18352] [18147]
18148 [    0     1     2 ... 18350 18351 18352] [18148]
18149 [    0     1     2 ... 18350 18351 18352] [18149]
18150 [    0     1     2 ... 18350 18351 18352] [18150]
18151 [    0     1     2 ... 18350 18351 18352] [18151]
18152 [    0     1     2 ... 18350 18351 18352] [18152]
18153 [    0     1     2 ... 18350 18351 18352] [18153]
18154 [    0     1     2 ... 18350 18351 18352] [18154]
18155 [    0     1     2 ... 18350 18351 18352] [18155]
18156 [    0     1     2 ... 18350 18351 18352] [18156]
18157 [    0     1     2 ... 18350 18351 18352] [18157]
18158 [    0     1     2 ... 18350 18351 18352] [18158]
18159 [    0     1     2 ... 18350 18351 18352] [18159]
18160 [    0     1     2 ... 18350 18351 18352] [18160]
18161 [    0     1     2 ... 18350 18351 18352] [18161]
18162 [    0     1     2 ... 18350 18351 18352] [18162]
18163 [    0     1     2 ... 18350 18351 18352] 

In [40]:
## shufflesplit
from sklearn.model_selection import ShuffleSplit
ssp = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
for train_idx, valid_idx in ssp.split(train_valid_data):
    print(train_idx, valid_idx)

[ 5798  7100 13011 ...  5192 12172   235] [12785  6270   201 ... 15693 16978  9203]
[ 8711 15574   105 ...  7334  5271  2071] [  373  3433 17289 ...  1577  6524 17695]
[  873  6821  1776 ...  6721 12754  9574] [11787 13717 14828 ...   879  6546 17862]
[ 2258 13832  4562 ... 10717 16357 17420] [13016  1048 17172 ...  9373  2412  1293]
[11193 11837  6989 ... 14699 14443 16100] [11517 17852  2653 ...  3991 16576  6324]


In [1]:
import pandas as pd
import os
import shutil

In [2]:
data = pd.read_csv('/Users/xiaoqiang/Mlearning/汽车分类/train.csv')

In [5]:
data

,image,label
0,11185,5
1,76332,3
2,16686,0
3,45978,5
4,92662,1
...,...,...
4495,35543,1
4496,47394,3
4497,65614,3
4498,16934,0


In [6]:
k_fold = ['fold1', 'fold2', 'fold3', 'fold4', 'fold5']

,image,label
0,11185,5
1,76332,3
2,16686,0
3,45978,5
4,92662,1
...,...,...
195,3003,4
196,44316,3
197,31440,1
198,9420,4


In [11]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
for train_idx, valid_idx in kfold.split(range(4500)):
    print(train_idx)
    print(len(train_idx))

[ 900  901  902 ... 4497 4498 4499]
3600
[   0    1    2 ... 4497 4498 4499]
3600
[   0    1    2 ... 4497 4498 4499]
3600
[   0    1    2 ... 4497 4498 4499]
3600
[   0    1    2 ... 3597 3598 3599]
3600


In [12]:
data

,image,label
0,11185,5
1,76332,3
2,16686,0
3,45978,5
4,92662,1
...,...,...
4495,35543,1
4496,47394,3
4497,65614,3
4498,16934,0


In [17]:
weight = data.groupby('label').count()

In [18]:
weight

,image
label,
0,579
1,556
2,572
3,584
4,611
5,583
6,604
7,411


In [40]:
# 权重取样
val_image = []
train_image = []
for i in range(8):
    new_data = data[data['label']==i]
    if i == 7:
        val_image.extend(new_data[:95]['image'])
        train_image.extend(new_data[95:]['image'])
    else:
        val_image.extend(new_data[:115]['image'])
        train_image.extend(new_data[115:]['image'])

In [42]:
len(val_image)

900

In [43]:
len(train_image)

3600

In [46]:
train_df = pd.DataFrame({'image':train_image, 'label':data[data['image']==i for i in train_image]})

SyntaxError: invalid syntax (2621017350.py, line 1)

In [21]:
import numpy as np
import torch
classes, counts = np.unique(data['label'], return_counts=True)
num_class = classes.shape[0]
count = counts.max()
weights = np.zeros(len(data))
for c in classes:
    freq_c = np.sum(data['label']==c)
    weights[data['label']==c] = len(data)/freq_c
weights = list(weights)
sampler = torch.utils.data.WeightedRandomSampler(weights, int(num_class*count))
len(sampler)

4888

In [30]:
list(index)

TypeError: 'NoneType' object is not iterable

In [47]:
data

,image,label
0,11185,5
1,76332,3
2,16686,0
3,45978,5
4,92662,1
...,...,...
4495,35543,1
4496,47394,3
4497,65614,3
4498,16934,0


In [48]:
val_df = data[:900]

In [52]:
train_df = data[900:]

In [53]:
train_df

,image,label
900,22016,2
901,4932,6
902,69011,3
903,69140,1
904,42889,3
...,...,...
4495,35543,1
4496,47394,3
4497,65614,3
4498,16934,0


In [55]:
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

In [56]:
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')

In [61]:
train =list(train_data['image'])
val = list(val_data['image'])

In [68]:
file_list = os.listdir('/Users/xiaoqiang/Mlearning/汽车分类/train_data')

In [69]:
file_list

['14147.jpg',
 '23933.jpg',
 '82643.jpg',
 '7078.jpg',
 '80732.jpg',
 '99263.jpg',
 '89070.jpg',
 '21856.jpg',
 '16022.jpg',
 '42315.jpg',
 '837.jpg',
 '32802.jpg',
 '7722.jpg',
 '98141.jpg',
 '1435.jpg',
 '83207.jpg',
 '30967.jpg',
 '94009.jpg',
 '18755.jpg',
 '36443.jpg',
 '64305.jpg',
 '5874.jpg',
 '79432.jpg',
 '79426.jpg',
 '48823.jpg',
 '22422.jpg',
 '65769.jpg',
 '37749.jpg',
 '28641.jpg',
 '46826.jpg',
 '30768.jpg',
 '25403.jpg',
 '23066.jpg',
 '45485.jpg',
 '23700.jpg',
 '84756.jpg',
 '94223.jpg',
 '61902.jpg',
 '98419.jpg',
 '77812.jpg',
 '5479.jpg',
 '14437.jpg',
 '13358.jpg',
 '91401.jpg',
 '63697.jpg',
 '36866.jpg',
 '36872.jpg',
 '28494.jpg',
 '86169.jpg',
 '97064.jpg',
 '95698.jpg',
 '90086.jpg',
 '29761.jpg',
 '3975.jpg',
 '82496.jpg',
 '86196.jpg',
 '62562.jpg',
 '31884.jpg',
 '77609.jpg',
 '38136.jpg',
 '21301.jpg',
 '6825.jpg',
 '63126.jpg',
 '3791.jpg',
 '27764.jpg',
 '43590.jpg',
 '2276.jpg',
 '4161.jpg',
 '14231.jpg',
 '12640.jpg',
 '85528.jpg',
 '27189.jpg',
 '80

In [78]:
os.makedirs('train_dir', exist_ok=True)
os.makedirs('val_dir', exist_ok=True)
for i in file_list:
    if int(i.split('.')[0]) in train:
        shutil.copy(os.path.join('/Users/xiaoqiang/Mlearning/汽车分类/train_data', i), 'train_dir')
    else:
        shutil.copy(os.path.join('/Users/xiaoqiang/Mlearning/汽车分类/train_data', i), 'val_dir')

In [70]:
len(file_list)

4500

In [74]:
int(file_list[0].split('.')[0])

14147

In [75]:
train[0]

22016

In [76]:
14147 in train

True